Goal of this notebook: generate an entry for the kaggle galaxy classification challenge

- [x] split out a validation set

- [x] parse the solution csv file

- [x] write a generator to find picture for each solution

- [x] use fit_generator to train model

In [47]:
import csv, random, os, bcolz, glob
import numpy as np
from scipy.ndimage import imread
from scipy.misc import imresize
import matplotlib.pyplot as plt
from vgg16 import Vgg16, Dense, Adam, Sequential
from utils import *

In [2]:
%matplotlib inline

In [3]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [4]:
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = LESSON_HOME_DIR + "/data/galaxy"
solutions_csv = DATA_HOME_DIR + "/solns.csv"

In [5]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()


def load_array(fname):
    return bcolz.open(fname)[:]

## make validation set

In [6]:
valid_array = load_array(DATA_HOME_DIR + "/valid/valid_solutions")
train_array = load_array(DATA_HOME_DIR + "/train/training_solutions")

In [7]:
print valid_array.shape
#print valid_array[:5, :]
#print valid_array[0, 1:]

(12159, 38)


In [8]:
train_ids = map(int, train_array[:,0])
valid_ids = map(int, valid_array[:,0])

## make samples

In [9]:
from shutil import copyfile

In [10]:
train_sample_size = 200
valid_sample_size = 100
test_sample_size = 200

## test an idea for generator

## make batched generators

In [11]:
def get_pic_by_id(uid, folder = DATA_HOME_DIR + "/train/"):
    return np.swapaxes(np.swapaxes(imresize(imread(folder + "%d.jpg" % uid), (224, 224)), 1, 2), 0, 1)

In [62]:
get_pic_by_id(train_ids[0]).shape

(3, 224, 224)

In [44]:
def img_gen(solnarray, batchsize, folder = DATA_HOME_DIR + "/train/"):
    imgen = (get_pic_by_id(rr[0], folder) for rr in list(solnarray))
    while(1):
        img_out = []
        for i in range(batchsize):
            try:
                this_img = imgen.next()
                img_out.append(this_img)
            except StopIteration:
                imgen = (get_pic_by_id(rr[0], folder) for rr in list(solnarray))
                this_img = imgen.next()
            
        all_img_out = np.stack(img_out, axis=0)
        yield all_img_out

In [13]:
def data_gen(solnarray, batchsize, folder = DATA_HOME_DIR + "/train/"):
    imgen = ((get_pic_by_id(rr[0], folder), rr[1:]) for rr in list(solnarray))
    while(1):
        img_out = []
        soln_out = []
        for i in range(batchsize):
            try:
                this_img, this_soln = imgen.next()
            except StopIteration:
                imgen = ((get_pic_by_id(rr[0], folder), rr[1:]) for rr in list(solnarray))
                this_img, this_soln = imgen.next()
            img_out.append(this_img)
            soln_out.append(this_soln)
        all_img_out = np.stack(img_out, axis=0)
        all_soln_out = np.stack(soln_out, axis=0)
        yield (all_img_out, all_soln_out)

## alter the vgg16 model

In [14]:
vgg = Vgg16()

In [15]:
hidden_size = 64
output_size = train_array.shape[1] - 1
lr = 0.001

model = vgg.model
model.pop()
for layer in model.layers: layer.trainable = False
model.add(Dense(hidden_size, activation = 'relu'))
model.add(Dense(output_size, activation = None))
model.compile(optimizer=Adam(lr=lr), loss='mse')

## precompute the cnn layers

In [16]:
pre_layers = model.layers[:-2]
pre_model = Sequential(pre_layers)
#pre_model.summary()

## train the model

In [80]:
batchsize = 64

vgg.model.fit_generator(generator = data_gen(train_array, batchsize),
                 samples_per_epoch = train_array.shape[0],
                 nb_epoch = 1,
                 validation_data = data_gen(valid_array, batchsize, folder = DATA_HOME_DIR + "/valid/"),
                 nb_val_samples = valid_array.shape[0])

Epoch 1/1
49472/49419 [==============================] - 1586s - loss: 0.0627 - val_loss: 0.0276


## generate test predictions